In [1]:
'''
Author        : Aditya Jain
Date started  : 10th January, 2022
About         : This script fine tunes the model with hard examples
'''

'\nAuthor        : Aditya Jain\nDate started  : 10th January, 2022\nAbout         : This script fine tunes the model with hard examples\n'

In [2]:
from comet_ml import Experiment

experiment = Experiment(
    api_key="epeaAhyRcHSkn92H4kusmbX8k",
    project_name="selfsupervision",
    workspace="adityajain07",
)

COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.ml https://www.comet.ml/adityajain07/selfsupervision/fb6c0eb26030426486ed72efe2def194



In [10]:
import torchvision
from torchvision import models
from torchvision import datasets
from torchvision import transforms, utils
from torchvision.transforms import ToTensor
from torch.utils.data import Dataset, DataLoader

import torch
from torch import nn
import torch.optim as optim
from torchsummary import summary
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import time
import datetime
import os

from data.imagenetvaldataset import ImagenetValDataset
from data.hardexdataset import HardExDataset

#### Loading pre-trained ResNet50 Model

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

model  = models.resnet50(pretrained=True).to(device)

cuda


#### Edits to make

In [5]:
val_root_dir       = '/network/datasets/imagenet.var/imagenet_torchvision/val/'
val_label_list     = '/home/mila/a/aditya.jain/mothAI/selfsupervision/data/validation_imagenet_labels.csv'
val_convert_list   = '/home/mila/a/aditya.jain/mothAI/selfsupervision/data/imagenet_modified_labels.csv'

hardex_data_dir    = '/home/mila/a/aditya.jain/scratch/selfsupervise_data/hard_examples/'
hardex_label_list  = '/home/mila/a/aditya.jain/mothAI/selfsupervision/data/hard_examples_data.csv'

image_resize       = 224
batch_size         = 32

DTSTR              = datetime.datetime.now()
DTSTR              = DTSTR.strftime("%Y-%m-%d-%H-%M")
mod_save_path      = '/home/mila/a/aditya.jain/logs/'
mod_name           = 'selfsupervisemodel'
mod_ver            = 'v1'
save_path          = mod_save_path + mod_name + '_' + mod_ver  + '_' + DTSTR + '.pt'
early_stop         = 4
epochs             = 30

transformer        = transforms.Compose([
                        transforms.Resize((image_resize, image_resize)),              # resize the image to 224x224 
                        transforms.ToTensor(),
                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
                                    ])

#### Evaluation on Validation Data

In [7]:
val_data          = ImagenetValDataset(val_root_dir, val_label_list, val_convert_list, transformer)
val_dataloader    = DataLoader(val_data,batch_size=batch_size)


In [7]:
def batch_accuracy(predictions, labels):
    '''
    calculates top1, top5 and top10 correct predictions in a batch
    '''
    top1         = 0
    top5         = 0
    top10        = 0
    
    _, pr_indices1  = torch.topk(predictions, 1)
    _, pr_indices5  = torch.topk(predictions, 5)
    _, pr_indices10 = torch.topk(predictions, 10)
    
    for i in range(len(labels)):
        if labels[i] in pr_indices1[i]:
            top1 += 1
        
        if labels[i] in pr_indices5[i]:
            top5 += 1
            
        if labels[i] in pr_indices10[i]:
            top10 += 1
            
    return top1, top5, top10
    

In [8]:
top1_correct  = 0
top5_correct  = 0
top10_correct = 0
total         = 0

model.eval()
for image_batch, label_batch in val_dataloader:
    
    image_batch, label_batch = image_batch.to(device), label_batch.to(device)
    predictions              = model(image_batch)    
    top1, top5, top10        = batch_accuracy(predictions, label_batch)
   
    top1_correct    += top1
    top5_correct    += top5
    top10_correct   += top10
    total           += len(label_batch)
    
pretrain_accuracy                   = {}
pretrain_accuracy['top1_pretrain_val']  = top1_correct/total*100
pretrain_accuracy['top5_pretrain_val']  = top5_correct/total*100
pretrain_accuracy['top10_pretrain_val'] = top10_correct/total*100

experiment.log_metrics(pretrain_accuracy)

#### Fine-tuning using hard examples

In [6]:
hardex_data        = HardExDataset(hardex_data_dir, hardex_label_list, transformer)
hardex_dataloader  = DataLoader(hardex_data, batch_size=batch_size, shuffle=True)

In [7]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
start_val_loss  = 10000000
lowest_val_loss = start_val_loss
early_stp_count = 0

for epoch in range(epochs):
    train_loss = 0
    val_loss   = 0
    s_time     = time.time()
    
    # model fine-tuning
    model.train()
    for image_batch, label_batch in hardex_dataloader:
        image_batch, label_batch = image_batch.to(device), label_batch.to(device)
        label_batch              = label_batch.squeeze_() 
        
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs   = model(image_batch)        
        t_loss    = loss_func(outputs, label_batch)
        t_loss.backward()
        optimizer.step()        
        train_loss += t_loss.item()
        
    experiment.log_metric("loss_train", train_loss, epoch=epoch)
    
    # model evaluation
    top1_correct  = 0
    top5_correct  = 0
    top10_correct = 0
    total         = 0

    model.eval()
    for image_batch, label_batch in val_dataloader:    
        image_batch, label_batch = image_batch.to(device), label_batch.to(device)
        label_batch              = label_batch.squeeze_() 
        predictions              = model(image_batch)
        
        v_loss    = loss_func(predictions, label_batch)
        val_loss += v_loss.item()
        
        top1, top5, top10  = batch_accuracy(predictions, label_batch)   
        top1_correct      += top1
        top5_correct      += top5
        top10_correct     += top10
        total             += len(label_batch)
    
    experiment.log_metric("loss_val", val_loss, epoch=epoch)
    experiment.log_metric('top1_tuned_val', top1_correct/total*100, epoch=epoch)
    experiment.log_metric('top5_tuned_val', top5_correct/total*100, epoch=epoch)
    experiment.log_metric('top10_tuned_val', top10_correct/total*100, epoch=epoch)
    
    e_time = (time.time()-s_time)/60      
    experiment.log_metric("time_per_epoch", e_time, epoch=epoch)
    
    if val_loss<lowest_val_loss:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'train_loss': train_loss,
            'val_loss':val_loss}, 
            save_path)               
        lowest_val_loss = val_loss
        early_stp_count = 0
    else:
        early_stp_count += 1 
        
    if early_stp_count>=early_stop:
        break 


#### Calculate class-wise accuracies

In [11]:
# only when running separately, otherwise the current model is used
PATH       = '/home/mila/a/aditya.jain/logs/selfsupervisemodel_v1_2022-01-30-23-00.pt'
checkpoint = torch.load(PATH, map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])

# changing batch size
batch_size        = 1 
val_data          = ImagenetValDataset(val_root_dir, val_label_list, val_convert_list, transformer)
val_dataloader    = DataLoader(val_data,batch_size=batch_size)

In [12]:
class_acc         = {}  # storing the class accuracy data

data_loc          = '/home/mila/a/aditya.jain/scratch/selfsupervise_data/hard_examples/'
hard_classes      = os.listdir(data_loc)

with open("/home/mila/a/aditya.jain/mothAI/selfsupervision/data/imagenet_classes.txt", "r") as f:
    categories = [s.strip() for s in f.readlines()]

In [14]:
model.eval()
for image_batch, label_batch in val_dataloader:
    
    image_batch, label_batch = image_batch.to(device), label_batch.to(device)
    prediction               = model(image_batch)    
    _, index                 = torch.topk(prediction, 1)
    
    corr_label   = label_batch.cpu().numpy()[0][0]    # integer label of the true class
    corr_class_n = categories[corr_label]             # name of the class
    pred_label   = index.cpu().numpy()[0][0]          # integer label of the predicted class
    
    if corr_class_n not in class_acc.keys():
        class_acc[corr_class_n] = {}
        class_acc[corr_class_n]['total_correct'] = 0
        class_acc[corr_class_n]['total_samples'] = 0
        
    if corr_label==pred_label:
        class_acc[corr_class_n]['total_correct'] += 1
        class_acc[corr_class_n]['total_samples'] += 1
    else:
        class_acc[corr_class_n]['total_samples'] += 1
        

In [ ]:
hard_ex_data    = []     # classes for which we have hard examples
nonhard_ex_data = []     # classes for which we don't have hard examples

for key in class_acc.keys():
    if key in hard_classes:
        hard_ex_data.append([categories.index(key), key, round((class_acc[key]['total_correct']/class_acc[key]['total_samples'])*100,2)])
    else:
        nonhard_ex_data.append([categories.index(key), key, round((class_acc[key]['total_correct']/class_acc[key]['total_samples'])*100,2)])

In [ ]:
pretrain_acc_data  = pd.read_csv(ILSVRC_LABEL_FILE)

In [ ]:
experiment.end()